In [1]:
%pip install llama_stack_client==0.3.0 fire docling

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 174.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 645.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 198.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 74.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 214.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 239.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 214.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 133.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 211.0 MB/s  0:00:04eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 190.8 MB/s  0:00:02eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 145.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from llama_stack_client import RAGDocument, LlamaStackClient
from docling.document_converter import DocumentConverter

/opt/app-root/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
client = LlamaStackClient(base_url="http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321")

In [4]:
models = client.models.list()
model_id = next(m.identifier for m in models if m.model_type == "llm")
embedding_model = next(m for m in models if m.model_type == "embedding")
embedding_model_id = embedding_model.identifier
embedding_dimension = int(embedding_model.metadata["embedding_dimension"])

INFO:httpx:HTTP Request: GET http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321/v1/models "HTTP/1.1 200 OK"


In [5]:
vector_db_name = "my_pgvector_db"

In [6]:
# Explicit - specify embedding model and/or provider when you need specific ones
vs = client.vector_stores.create(
    name=vector_db_name,
    extra_body={
    "provider_id": "pgvector", # Optional: specify vector store provider
    "embedding_model": embedding_model_id,
    "embedding_dimension": embedding_dimension # Optional: will be auto-detected if not provided
    }
)
print(f"Registered remote pgvector DB: {vs.id}")
vector_db_id = vs.id

INFO:httpx:HTTP Request: POST http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321/v1/vector_stores "HTTP/1.1 200 OK"


Registered remote pgvector DB: vs_6fb2c6fd-2a11-48f6-9a88-ea9b66b13e8c


In [7]:
# RAW Text Ingestion
raw_text = """
LlamaStack can embed raw text into a vector store for retrieval.
This example ingests a small passage for demonstration.
"""
document = RAGDocument(
    document_id="text_doc_id_001",
    content=raw_text,
    mime_type="text/plain",
    metadata={"source": "example_passage"},
)
client.tool_runtime.rag_tool.insert(
    documents=[document],
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=100,
)
print("Raw text ingested successfully")

INFO:httpx:HTTP Request: POST http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321/v1/tool-runtime/rag-tool/insert "HTTP/1.1 200 OK"


Raw text ingested successfully


In [8]:
# HTML Ingestion
source = "https://www.paulgraham.com/greatwork.html"
document = RAGDocument(
    document_id="html_doc_id_001",
    content=source,
    mime_type="text/html",
    metadata={},
)
client.tool_runtime.rag_tool.insert(
    documents=[document],
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=50,
)
print("HTML ingested successfully")

INFO:httpx:HTTP Request: POST http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321/v1/tool-runtime/rag-tool/insert "HTTP/1.1 200 OK"


HTML ingested successfully


In [9]:
# PDF as Markdown Ingestion
# 1. Initialize the converter
source = "./red_hat_openshift_ai_self-managed-2.16-getting_started_with_red_hat_openshift_ai_self-managed-en-us.pdf"
converter = DocumentConverter()

# 2. Convert the PDF
# This step handles layout analysis, table extraction, etc.
result = converter.convert(source)

# 3. Export to Markdown for the best RAG performance
markdown_content = result.document.export_to_markdown()

# 4. Ingest into your RAGDocument
document = RAGDocument(
    document_id="pdf_as_markdown_doc_id_001",
    content=markdown_content,
    mime_type="text/markdown",  # Note: Use markdown for better LLM reasoning
    metadata={
        "source": source,
        "page_count": len(result.document.pages)
    },
)
client.tool_runtime.rag_tool.insert(
    documents=[document],
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=512,
)
print("PDF as Markdown ingested successfully")

INFO:docling.datamodel.document:detected formats: [<InputFormat.PDF: 'pdf'>]
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Initializing pipeline for StandardPdfPipeline with options hash e15bc6f248154cc62f8db15ef18a8ab7
INFO:docling.models.factories.base_factory:Loading plugin 'docling_defaults'
INFO:docling.models.factories:Registered picture descriptions: ['vlm', 'api']
INFO:docling.models.factories.base_factory:Loading plugin 'docling_defaults'
INFO:docling.models.factories:Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
INFO:docling.models.auto_ocr_model:rapidocr cannot be used because onnxruntime is not installed.
INFO:docling.models.auto_ocr_model:easyocr cannot be used because it is not installed.
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
[INFO] 2026-01-17 22:16:38,882 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-17 22:16:38,885 [RapidOCR] devic

PDF as Markdown ingested successfully
